In [1]:
import numpy as np
import sympy

import plotly
import plotly.graph_objects as go
import plotly.express as px

In [71]:
def curvature_orthogonal_monge(Z, spacing=None):
    """
    Z is a 2D array
    This assumes that your data points are equal units apart

    The matrix Z is a 2D array with vertices arranged in the mesh form:

    O--O--O--O
    |  |  |  |
    O--O--O--O
    |  |  |  |
    O--O--O--O
    """
    # https://stackoverflow.com/questions/11317579/surface-curvature-matlab-equivalent-in-python

    (lr, lb) = Z.shape

    Zy, Zx = np.gradient(Z)
    Zxy, Zxx = np.gradient(Zx)
    Zyy, _ = np.gradient(Zy)

    Zy = Zy / spacing
    Zx = Zx / spacing
    Zxy = Zxy / spacing**2
    Zxx = Zxx / spacing**2
    Zyy = Zyy / spacing**2
    

    Zx = np.reshape(Zx, lr * lb)
    Zy = np.reshape(Zy, lr * lb)
    Zxx = np.reshape(Zxx, lr * lb)
    Zxy = np.reshape(Zxy, lr * lb)
    Zyy = np.reshape(Zyy, lr * lb)

    # Gaussian curvature
    K = (Zxx * Zyy - (Zxy**2)) / (1 + (Zx**2) + (Zy**2)) ** 2
    # Mean curvature
    H = (Zx**2 + 1) * Zyy - 2 * Zx * Zy * Zxy + (Zy**2 + 1) * Zxx
    H = H / (2 * (Zx**2 + Zy**2 + 1) ** (1.5))

    n = 1/np.sqrt(1+Zx**2+Zy**2)

    E = 1+Zx**2
    F = Zx*Zy
    G = 1+Zy**2

    L = Zxx*n
    M = Zxy*n
    N = Zyy*n

    print(f'E[0] {E[0]}')
    print(f'F[0] {F[0]}')
    print(f'G[0] {G[0]}')
    print(f'L[0] {L[0]}')
    print(f'M[0] {M[0]}')
    print(f'N[0] {N[0]}')
    

    # for local parametrization f : V → S 
    # FF = np.array([[L, M], [M, N]]) # First fundamental form
    # SFF = np.array([[E, F], [F, G]]) # Second fundamental form
    # # # reshape so that the 2D matrices are in the ros/cols (lr*lb, 2, 2)
    # FFF = np.swapaxes(FFF, 0, 2)
    # SFF = np.swapaxes(SFF, 0, 2)
    # print("FFF shape", FFF.shape)
    # print("SFF shape", SFF.shape)
    # print(f'FFF[0] {FFF[0]}')
    # print(f'SFF[0] {SFF[0]}')
    # # # print("FFF", FFF)
    # P = FFF * np.linalg.inv(SFF)


    # TODO: Shape operator
    P = np.array(
        [
            [
                Zxx * (1 + Zy**2) - Zxy * Zx * Zy,
                Zxy * (1 + Zx**2) - Zxx * Zx * Zy,
            ],
            [
                Zxy * (1 + Zy**2) - Zyy * Zx * Zy,
                Zyy * (1 + Zx**2) - Zxy * Zx * Zy,
            ],
        ]
    ) / ((1 + Zx**2 + Zy**2) ** (3 / 2))
    # reshape so that the 2D matrices are in the ros/cols (lr*lb, 2, 2)
    P = np.swapaxes(P, 0, 2)
    P = np.swapaxes(P, 2, 1)


    print("P shape", P.shape)

    X = np.linalg.eig(P)
    # the result of eig is a tuple of (eigenvalues, eigenvectors)
    # print(f'X {X.shape} {X}')
    print(f'X length {len(X)}')
    # print(f'X[0] {X[0].shape} {X[0]}')
    print(f'X[1] {X[1].shape} {X[1]}')
    k1 = X[0][:, 0]  # all the first eigenvalues
    k2 = X[0][:, 1]  # all the second eigenvalues
    X1 = X[1][:, :, 0]  # all the first eigenvectors
    X2 = X[1][:, :, 1]  # all the second eigenvectors

    # add a dimension to the end (lr*lb, 2, 1)
    X1 = np.expand_dims(X1, 2)  
    X2 = np.expand_dims(X2, 2)
    print(f'X1.shape {X1.shape}')
    # print(X1)

    # ensure Zx,Zy are in column form
    Zx = np.reshape(Zx, (lr*lb, 1))
    Zy = np.reshape(Zy, (lr*lb, 1))

    # create df in parametric form which equals [[[1,0],[0,1],[Zx,Zy]],...]
    top = np.array([[[spacing, 0], [0, spacing]]])
    top = np.repeat(top, repeats=lr*lb, axis=0)
    dX = np.dstack((Zx, Zy))
    # print(f'dX shape {dX.shape}')
    dX = np.hstack((top, dX))
    # dX should now be in shape (lr*lb,3,2)
    print(f'dX shape {dX.shape}')
    print(dX)

    # matrix multiplication of dX and X for each point
    k1vec = np.einsum("lij,ljk->lik", dX, X1)
    k2vec = np.einsum("lij,ljk->lik", dX, X2)
    print(f'k1vec shape {k1vec.shape}')

    # normalize the vectors
    # the vectors are currently in shape (lr*lb, 3, 1)
    k1vec = k1vec / np.linalg.norm(k1vec, axis=1, keepdims=True)
    k2vec = k2vec / np.linalg.norm(k2vec, axis=1, keepdims=True)

    ## alternatively
    k1 = H + np.sqrt(H**2 - K)
    k2 = H - np.sqrt(H**2 - K)

    K = np.reshape(K, (lr, lb))
    H = np.reshape(H, (lr, lb))
    k1 = np.reshape(k1, (lr, lb))
    k2 = np.reshape(k2, (lr, lb))
    k1vec = np.reshape(k1vec, (lr, lb, 3)) # (lr, lb, 3, 1)
    k2vec = np.reshape(k2vec, (lr, lb, 3))


    return K, H, k1, k2, k1vec, k2vec



In [64]:
u, v = sympy.symbols("u v")
# f_explicit = sympy.sqrt(100 - v**2 - u**2)
# f_explicit = sympy.sqrt(100 - v**2 -2*u**2)
# f_explicit = sympy.sqrt(100 - (0.5*v - 0.5*u))
f_explicit = 2 + -(2*u-v)*(2*u-v)
x, y = u, v

# coordinate range
xx = np.linspace(-1, 1, 20)
yy = np.linspace(-1, 1, 20)

# make coordinate point
X, Y = np.meshgrid(xx, yy)

# dependent variable point on coordinate
f2 = sympy.lambdify((x, y), f_explicit)
Z = f2(X, Y)

fig = go.Figure(go.Surface(x=X, y=Y, z=Z, surfacecolor=Z)) # np.zeros_like(Z)
fig.show()

In [72]:
K, H, k1, k2, k1vec, k2vec = curvature_orthogonal_monge(Z, spacing=0.1)

print('k1[10,10]', k1[10,10])
print('k2[10,10]', k2[10,10])
# print(k1vec[10,10])
# print(k2vec[10,10])
print('k1vec * k2vec', np.dot(k1vec[10,10], k2vec[10,10]))
print('K[10,10]', K[10,10])
print('H[10,10]', H[10,10])

# print(k1vec[1,1])
# print(k2vec[1,1])
# print(np.dot(k1vec[1,1], k2vec[1,1]))

# fig = px.imshow(k2, labels=dict(x="X", y="Y", color="k2"),)


fig = go.Figure(
    data=[
        go.Cone(
            x=X.ravel(),
            y=Y.ravel(),
            z=Z.ravel(),
            u=k2vec[:, :, 0].ravel(),
            v=k2vec[:, :, 1].ravel(),
            w=k2vec[:, :, 2].ravel(),
            # u=k2vec[:, 0].ravel(),
            # v=k2vec[:, 1].ravel(),
            # w=k2vec[:, 2].ravel(),
            # sizemode="absolute",
            # sizeref=2,
            anchor="tip",
        ),
        go.Cone(
            x=X.ravel(),
            y=Y.ravel(),
            z=Z.ravel(),
            u=k1vec[:, :, 0].ravel(),
            v=k1vec[:, :, 1].ravel(),
            w=k1vec[:, :, 2].ravel(),
            # sizemode="absolute",
            # sizeref=1,
            anchor="tip",
        ),
    ]
)
fig.add_trace(go.Surface(x=X, y=Y, z=Z, surfacecolor=H)) # , surfacecolor=H
fig.show()

E[0] 15.192647386069778
F[0] -8.348616109452799
G[0] 5.910950652619286
L[0] -0.9884982041876289
M[0] 0.988498204187619
N[0] -0.24712455104691466
P shape (400, 2, 2)
X length 2
X[1] (400, 2, 2) [[[-0.85412713 -0.73681545]
  [ 0.52006427 -0.67609392]]

 [[-0.87965318 -0.5450416 ]
  [ 0.47561569 -0.838409  ]]

 [[-0.89923419 -0.11352438]
  [ 0.43746756 -0.99353521]]

 ...

 [[-0.89923419 -0.11352438]
  [ 0.43746756 -0.99353521]]

 [[-0.87965318 -0.5450416 ]
  [ 0.47561569 -0.838409  ]]

 [[-0.85412713 -0.73681545]
  [ 0.52006427 -0.67609392]]]
X1.shape (400, 2, 1)
dX shape (400, 3, 2)
[[[ 0.1         0.        ]
  [ 0.          0.1       ]
  [ 3.76731302 -2.21606648]]

 [[ 0.1         0.        ]
  [ 0.          0.1       ]
  [ 3.32409972 -1.77285319]]

 [[ 0.1         0.        ]
  [ 0.          0.1       ]
  [ 2.43767313 -1.32963989]]

 ...

 [[ 0.1         0.        ]
  [ 0.          0.1       ]
  [-2.43767313  1.32963989]]

 [[ 0.1         0.        ]
  [ 0.          0.1       ]
  [-3

In [5]:
a = np.array([[[1,1,1], [2,2,2]]])
a.shape # (1, 3)
# (lr, lb) = a.shape
# a = np.reshape(a, (1,1, lr * lb))
# a.shape # (1, 3)
# c = np.swapaxes(a, 0, 2)
# c
# c.shape
# a

(1, 2, 3)